In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo import UpdateOne

# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))
cl_udrt_kakao_cdh=db['udrt_kakao_cdh']
cl_udrt_kakao_cdh_updated2 = db['udrt_kakao_cdh_updated2']

In [ ]:
# udrt_kakao_cdh_update2에서 _id가 일치하는 문서를 udrt_kakao_cdh에서 찾아서 'road_address_name' 추가
cl_udrt_kakao_cdh_dict={doc["_id"]: doc["road_address_name"] for doc in cl_udrt_kakao_cdh.find({},{"_id": 1, "road_address_name": 1})}

bulk_updates=[]
for doc in cl_udrt_kakao_cdh_updated2.find():
    doc_id=doc["_id"]

    road_address_name=cl_udrt_kakao_cdh_dict.get(doc_id)
    bulk_updates.append({
            "filter": {"_id": doc_id},
            "update": {"$set": {"road_address_name": road_address_name}}
        })  
if bulk_updates:
    cl_udrt_kakao_cdh_updated2.bulk_write([UpdateOne(update["filter"], update["update"]) for update in bulk_updates])
print(f"{len(bulk_updates)}개의 문서를 업데이트 완료!")
    

151176개의 문서를 업데이트 완료!
